In [1]:
import pandas as pd
import numpy as np

import json

In [2]:
bids = pd.read_csv('../data/bid_requests.csv')
users = pd.read_csv('../data/user_attributes.csv')

In [3]:
bids.head()

,timestamp,user_id,bid,win,conversion
0,2017-01-01 00:00:01,be7485be5b6eb3690efcbc9e95e8f15a,0,0,0
1,2017-01-01 00:00:02,26c5dca2512a4c7fe8810bd04191b1b3,0,0,0
2,2017-01-01 00:00:05,2121376a323507c01c5e92c39ae8ccd4,0,0,0
3,2017-01-01 00:00:12,fa6a0925d911185338b0acc93c66dc92,0,0,0
4,2017-01-01 00:00:13,4299f209da83da82b711f1d631cc607b,1,0,0


In [4]:
users.head()

,user_id,attributes
0,00003e3b9e5336685200ae85d21b4f5e,"{""age"": 33, ""gender"": ""F"", ""location"": ""FL"", ""..."
1,000053b1e684c9e7ea73727b2238ce18,"{""age"": 26, ""gender"": ""M"", ""location"": ""AL"", ""..."
2,00029153d12ae1c9abe59c17ff2e0895,"{""age"": 29, ""gender"": ""F"", ""location"": ""AR"", ""..."
3,0002ac0d783338cfeab0b2bdbd872cda,"{""age"": 29, ""gender"": ""M"", ""location"": ""SC"", ""..."
4,0004d0b59e19461ff126e3a08a814c33,"{""age"": 27, ""gender"": ""F"", ""location"": ""AR"", ""..."


In [5]:
txt = users.attributes[1]

In [6]:
dic = json.loads(txt)

In [7]:
atts = pd.DataFrame(columns=['age', 'gender', 'location', 'test'])

In the interest of time, I've commented out the next two cells: the first cell creates a dataframe from the attributes textblob in the users dataframe. That process is extremely slow, so, in the following cell, I save that attributes dataframe as a csv, so we can just read it in whenever we run the notebook. 

In [8]:
# for i in range(len(users.attributes)):
#     dic = json.loads(users.attributes[i])
    
#     for k in dic:
#         dic[k] = [dic[k]]
        
#     df = pd.DataFrame.from_dict(dic)
    
#     atts = atts.append(df, ignore_index=True)
    
        

In [11]:
# atts.to_csv('../data/atts.csv', index=False)

In [12]:
atts = pd.read_csv('../data/atts.csv')

In [9]:
atts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   age       100000 non-null  object
 1   gender    100000 non-null  object
 2   location  100000 non-null  object
 3   test      100000 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [10]:
atts.head()

,age,gender,location,test
0,33,F,FL,1
1,26,M,AL,1
2,29,F,AR,1
3,29,M,SC,0
4,27,F,AR,1


In [13]:
atts.head()

,age,gender,location,test
0,33,F,FL,1
1,26,M,AL,1
2,29,F,AR,1
3,29,M,SC,0
4,27,F,AR,1
